In [1]:
import tkinter as tk
import serial
import time

In [3]:
arduino = serial.Serial("COM3", 9600)
time.sleep(2)

# Función para encender el flujo de agua
def encender():
    arduino.write(b'a')

# Función para apagar el flujo de agua    
def apagar():
    arduino.write(b'q')
   
# Creación de la ventana principal
window = tk.Tk()
window.title("Control de la bomba de agua")
window.geometry("300x150")
window.configure(bg="lightgray") # Color de fondo

bttnEncender = tk.Button(window, text="Encender", command=encender, bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
bttnEncender.place(x=50, y=75)

bttnApagar = tk.Button(window, text="Apagar", command=apagar, bg="white", fg="black", font=("Arial", 12, "bold"), bd=3)
bttnApagar.place(x=200, y=75)

window.mainloop()